# Pull all features of a given type for an area

In [ ]:
import numpy as np
import pandas as pd
import pyproj
import shapely
import osmnx

import plotly
from plotly.subplots import make_subplots
from plotly.graph_objects import Scatter

from geo_encodings import draw_shape

import polars as pl
pl.Config.set_tbl_rows(25)


In [ ]:
# Portsmouth NH.
lat0, lon0 = 43.043017, -70.790616
lat1, lon1 = 43.089772, -70.695498


In [ ]:
# Handy derived stuff.
query_bounds = [lon0, lat0, lon1, lat1]
center_lon = (lon0 + lon1) / 2
center_lat = (lat0 + lat1) / 2


In [ ]:
# Define a local map projection
offset = 20000
proj_def = f"""
+proj=tmerc +lat_0={center_lat} +lon_0={center_lon} 
+k=1.0 +x_0={offset} +y_0={offset} +datum=WGS84 +units=m +no_defs
"""
ltm_crs = pyproj.CRS.from_proj4(proj_def)
wgs84_crs = pyproj.CRS.from_epsg(4326)
proj_forward = pyproj.Transformer.from_crs(wgs84_crs, ltm_crs, always_xy=True).transform
proj_inverse = pyproj.Transformer.from_crs(ltm_crs, wgs84_crs, always_xy=True).transform


In [ ]:
tags = {
    'landuse': True,
    'place': True,
    'highway': True,
    'railway': True,
    'aeroway': True,
    'bridge': True,
    'tunnel': True,
    'power': True,
    'mannade': True,
    'barrier': True,
    'natural': True,
    'waterway': True,
    'landcover': True,
    'building': True,
    'amenity': True,
    'shop': True,
    'healthcare': True,
    'leisure': True
}
features = osmnx.features.features_from_bbox(query_bounds, tags=tags).reset_index()
print('%d features' % len(features))

In [ ]:
for c in sorted(features.columns):
    print(c)